In [1]:
# importing the libraries
import pandas as pd
import artm
import math
from tqdm import tqdm

In [2]:
# uploading data & picking only text part
df = pd.read_csv(r'/Users/apple/Documents/BDML /НИР/train.csv')
dataset1 = df['text']

# creating the function for transformation to vowpal_wabbit format


def df_to_vw_regression(df, filepath='in.txt', columns=None, target=None, namespace='namespace'):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write('{0} |{1} '.format(row[target], namespace))
            else:
                f.write('{0} | '.format(row[target]))
            last_feature = row.index.values[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                elif isinstance(val, float) or isinstance(val, int):
                    if not math.isnan(val):
                        f.write('{0}:{1}'.format(idx.replace(' ', '_').replace(':', '_'), val))
                    else:
                        continue
                else:
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')


def df_to_vw_classification(
        df,
        filepath='mc.txt',
        columns=None,
        target=None,
        tag=None,
        namespace='n'
):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)
    columns.remove(tag)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write(f"{row[target]} \'{row[tag]} |{namespace} ")
            else:
                f.write(f"{row[target]} \'{row[tag]} | ")
            last_feature = columns[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                elif isinstance(val, float) or isinstance(val, int):
                    if not math.isnan(val):
                        f.write(f"{idx.replace(' ', '_').replace(':', '_')}:{val}")
                    else:
                        continue
                else:
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')

In [17]:
# changing the type of data created
vw = df_to_vw_regression(df, filepath='data.txt', target='text')
# vw = df_to_vw_classification(df, filepath='data.txt', target='target')

73987it [00:19, 3705.51it/s]


In [18]:
# batching data for applying it to our model


batch_vectorizer = artm.BatchVectorizer(data_path='data.txt',
                                        data_format='vowpal_wabbit',
                                        collection_name='vw',
                                        target_folder='koselniy_batches')

# LDA (BigARTM package) Model

In [19]:
# setting up lda parameters


lda = artm.LDA(num_topics=15,
               alpha=10.1, beta=0.1,
               cache_theta=True,
               num_document_passes=2,
               dictionary=batch_vectorizer.dictionary)
#Phi is the ‘parts-versus-topics’ matrix, and theta is the ‘composites-versus-topics’ matrix
lda.fit_offline(batch_vectorizer=batch_vectorizer,
                num_collection_passes=10)

#checking up the parametrs of the matricies
lda.sparsity_phi_last_value
lda.sparsity_theta_last_value
lda.perplexity_last_value

1.2579537630081177

In [20]:
#top tokens for each class of all the clusters we've got
top_tokens = lda.get_top_tokens(num_tokens=200)
for i, token_list in enumerate(top_tokens):
    print('Topic #{0}: {1}'.format(i, token_list))

d = pd.DataFrame(top_tokens)
top_tokens_vec= d.apply(lambda x: ' '.join(x), axis=1)

Topic #0: ['человека', 'лет', 'жизни', 'года', 'год', 'цена', 'сегодня', 'жизнь', 'людей', 'работа', 'других', 'работы', 'сколько', 'день', 'времени', 'дня', 'любви', 'фото', 'самый', 'друга', 'хотя', 'состояние', 'несколько', 'должен', 'стороны', 'любовь', 'каждый', 'какие', 'первый', 'снова', 'дело', 'рядом', 'руб', 'лишь', 'мире', 'деньги', 'проблемы', 'рублей', 'часть', 'знает', 'состоянии', 'кроме', 'отдам', 'города', 'самых', 'хочу', 'полностью', 'мама', 'месте', 'новый', 'прямо', 'завтра', 'сентября', 'места', 'делать', 'счастье', 'июля', 'дома', 'помощью', 'момент', 'говорит', 'июня', 'поэтому', 'история', 'мая', 'родился', 'имя', 'образом', 'размер', 'сердце', 'самые', 'вместе', 'необходимо', 'дела', 'силы', 'россии', 'например', 'сша', 'москве', 'девушка', 'марта', 'жить', 'другие', 'мысли', 'почему', 'знаю', 'цвет', 'любит', 'работать', 'сил', 'кажется', 'году', 'новая', 'работе', 'скоро', 'пару', 'вашего', 'обычно', 'пожалуйста', 'тел', 'жена', 'пол', 'думаю', 'хотите', 'ин

# ARTM (BigARTM package) Model

In [21]:
#creating the batches for the second model and setting up bigartm dictionary
batch_vectorizer2 = artm.BatchVectorizer(data_path='data.txt',
                                         data_format='vowpal_wabbit',
                                         collection_name='vw',
                                         target_folder='koselniy_batches')
dictionary = batch_vectorizer2.dictionary

In [22]:
topic_names = ['topic_{}'.format(i) for i in range(15)]
#inial objects cration
model_plsa = artm.ARTM(topic_names=topic_names, cache_theta=True,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)])
model_artm = artm.ARTM(topic_names=topic_names, cache_theta=True,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta',
                                                                       tau=-0.15)])

In [23]:
#adding some scores for our future model
model_plsa.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_plsa.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_plsa.scores.add(artm.TopicKernelScore(name='TopicKernelScore',probability_mass_threshold=0.3))
model_plsa.scores.add(artm.TopTokensScore(name='Top_words', num_tokens=15, class_id='text'))
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopicKernelScore(name='TopicKernelScore',probability_mass_threshold=0.3))
model_artm.scores.add(artm.TopTokensScore(name='Top_words', num_tokens=15, class_id='text'))
model_plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=200))
model_artm.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=200))

#regulizers
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1))
model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=0.01))

model_plsa.num_document_passes = 1
model_artm.num_document_passes = 1

#initializing the model we've set up
model_plsa.initialize(dictionary=dictionary)
model_artm.initialize(dictionary=dictionary)

In [24]:
#fitting the model
#model_plsa.fit_offline(batch_vectorizer=batch_vectorizer2, num_collection_passes=15)
model_artm.fit_offline(batch_vectorizer=batch_vectorizer2, num_collection_passes=10)

In [25]:
#pickling te new model

In [26]:
#checking out the sparsity of our matricies
#print(model_artm.score_tracker["SparsityPhiScore"].last_value)
#print(model_artm.score_tracker["SparsityThetaScore"].last_value)

print(model_artm.score_tracker["SparsityPhiScore"].last_value)
print(model_artm.score_tracker["SparsityThetaScore"].last_value)

#checking out topic kernel score
print(model_artm.score_tracker["TopicKernelScore"])

#print(model_plsa.score_tracker["TopicKernelScore"])

0.7198261022567749
0.0


In [27]:
#checking up the top words
#print(model_artm.score_tracker["Top_words"])
print(model_artm.score_tracker["Top_words"])
#print(model_artm.)

In [28]:
for topic_name in model_artm.topic_names:
    print(topic_name + ': '),
    print(model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0: 
['человека', 'жизни', 'людей', 'лет', 'жизнь', 'сегодня', 'любви', 'года', 'руб', 'любовь', 'рядом', 'других', 'любить', 'год', 'сколько', 'лишь', 'день', 'времени', 'цена', 'работа', 'каждый', 'самый', 'сердце', 'снова', 'дня', 'значит', 'друга', 'работы', 'счастье', 'должен', 'какие', 'хотя', 'хочешь', 'фото', 'жить', 'стороны', 'первый', 'самые', 'девушка', 'деньги', 'несколько', 'поэтому', 'проблемы', 'делать', 'дело', 'слова', 'мире', 'любит', 'знает', 'кроме', 'завтра', 'дела', 'чувства', 'твои', 'никто', 'момент', 'другие', 'состояние', 'силы', 'хочу', 'хочется', 'самых', 'слишком', 'кажется', 'почему', 'работу', 'месте', 'мысли', 'твоя', 'сказать', 'мама', 'города', 'достаточно', 'полностью', 'имя', 'говорит', 'необходимо', 'вместе', 'часть', 'говорить', 'рублей', 'дома', 'новый', 'важно', 'смотреть', 'помощью', 'твоей', 'отношения', 'знаю', 'мая', 'меньше', 'хочет', 'бог', 'например', 'долго', 'состоянии', 'пор', 'постоянно', 'прямо', 'места', 'сил', 'отдам', 'июля',

['жизнь', 'люди', 'любовь', 'сегодня', 'день', 'года', 'жизни', 'хочется', 'друг', 'слова', 'работы', 'хочешь', 'цена', 'людям', 'новый', 'именно', 'году', 'стать', 'продам', 'никто', 'хочет', 'детей', 'человека', 'жить', 'самое', 'делать', 'людей', 'руб', 'твои', 'рублей', 'дом', 'например', 'фото', 'постоянно', 'деньги', 'слишком', 'пожалуйста', 'несколько', 'почему', 'россии', 'какие', 'мужчина', 'является', 'сделать', 'вопрос', 'внутри', 'проблемы', 'сразу', 'быстро', 'мысли', 'равно', 'лет', 'глаза', 'сердце', 'женщина', 'твоя', 'любой', 'обычно', 'других', 'дети', 'хочу', 'мира', 'раньше', 'любят', 'плохо', 'говорят', 'никому', 'снова', 'сказал', 'дома', 'получить', 'отношения', 'подскажите', 'год', 'пару', 'минут', 'людьми', 'должна', 'хотя', 'сентября', 'вещи', 'нужна', 'должен', 'руки', 'души', 'работу', 'важно', 'правильно', 'момент', 'особенно', 'мало', 'мама', 'некоторые', 'июня', 'дней', 'здравствуйте', 'писать', 'пути', 'утром', 'состояние', 'две', 'возможность', 'душе', 

['люди', 'жизни', 'года', 'сегодня', 'лишь', 'людей', 'руб', 'хочу', 'жизнь', 'почему', 'именно', 'делать', 'цена', 'никто', 'любишь', 'жить', 'первый', 'найти', 'нужен', 'должны', 'значит', 'продам', 'хочется', 'сразу', 'женщина', 'хочешь', 'вместе', 'ваши', 'мире', 'достаточно', 'другие', 'стал', 'году', 'вашей', 'сказать', 'сколько', 'новый', 'ночь', 'сделать', 'любовь', 'кроме', 'человека', 'деньги', 'выбираем', 'особенно', 'год', 'боль', 'минут', 'хотя', 'несколько', 'группу', 'равно', 'ради', 'назад', 'самых', 'важно', 'твоей', 'случае', 'внутри', 'фото', 'деле', 'обязательно', 'некоторые', 'времени', 'свет', 'следует', 'подскажите', 'любить', 'участников', 'запись', 'дома', 'девушка', 'среди', 'вопрос', 'вещи', 'человеком', 'например', 'рядом', 'город', 'сша', 'самые', 'каждый', 'душу', 'руки', 'чувства', 'любви', 'часто', 'телефон', 'имя', 'пути', 'мир', 'ждать', 'имеет', 'воды', 'размер', 'домой', 'действительно', 'многие', 'слишком', 'меньше', 'работать', 'любят', 'сердце', '

In [30]:
artmTokens = model_artm.score_tracker['TopTokensScore'].last_tokens
artmTokens = pd.DataFrame(artmTokens)
artmToks= artmTokens.apply(lambda x: ' '.join(x), axis=0)
artmToks

topic_0     человека жизни людей лет жизнь сегодня любви г...
topic_1     день сегодня лишь стоит люди жизни лет мир люб...
topic_2     день жизни люди людей каждый сколько лет любов...
topic_3     человека лет каждый хочу рядом людей сколько ф...
topic_4     жизнь жизни день человека сегодня рядом значит...
topic_5     жизнь стоит лишь жизни люди хочу года день дру...
topic_6     жизнь люди любовь сегодня день года жизни хоче...
topic_7     день жизнь людей люди лишь почему жизни хочу л...
topic_8     жизнь руб человека лет день года стоит мир люб...
topic_9     жизни люди человека людей почему поэтому девуш...
topic_10    люди сегодня года людей любовь лет стоит челов...
topic_11    жизни каждый люди лишь лет день любовь люблю с...
topic_12    люди жизни года сегодня лишь людей руб хочу жи...
topic_13    день жизни каждый лишь мир людям нужен сделать...
topic_14    жизнь людей люди сегодня день человека любовь ...
dtype: object

# KMeans model

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

/Users/apple/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


In [4]:
# retrieving text: setting up stop words, making Tf-Idf
words = stopwords.words('russian')
vectorizer = TfidfVectorizer(stop_words=words)
X = vectorizer.fit_transform(dataset1)

In [5]:
# counting KMeans for 13 clusters and fitting vectorized data in to the model
true_k = 20
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=20, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [6]:
#figuring out centroids for our data
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
centers = model.cluster_centers_

In [8]:
for i in range(true_k):
    print('Cluster %d:' % i),
    for ind in order_centroids[i, :30]:
        print(' %s' % terms[ind])

Cluster 0:
 руб
 цена
 продам
 наличии
 грн
 рублей
 размер
 тел
 состоянии
 продаю
 торг
 личку
 обмен
 состояние
 квартира
 пишите
 фото
 квартиру
 комплект
 доставка
 размеры
 iphone
 сатылады
 срочно
 куплю
 заказ
 звоните
 дом
 вопросы
 стоимость
Cluster 1:
 петли
 петель
 вязать
 сбн
 лиц
 ряд
 изн
 спицы
 петлю
 ряда
 провязать
 накид
 лицевой
 вяжем
 вязания
 узор
 ряду
 ряды
 лицевые
 спицами
 закрыть
 узором
 рядов
 набрать
 горловины
 гладью
 каждом
 вязание
 нить
 крючком
Cluster 2:
 работы
 график
 требуется
 требуются
 работа
 оплата
 требования
 опыт
 работу
 обязанности
 тел
 условия
 руб
 срочно
 ищу
 лет
 телефону
 обучение
 заработная
 плата
 день
 зарплата
 продавец
 трудоустройство
 работать
 пишите
 официальное
 оформление
 опыта
 месяц
Cluster 3:
 здравствуйте
 сколько
 стоить
 добрый
 стоимость
 подскажите
 пожалуйста
 день
 стоит
 цена
 скажите
 интересует
 вечер
 заказать
 цену
 личку
 замена
 узнать
 фото
 торт
 свадьба
 свадебная
 фотосессия
 тату
 макияж
 с

# RuBETR classifier

In [16]:
mat = dataset.values
labels = model.labels_

results = pandas.DataFrame([dataset.index,labels]).T
print(results)

[ 2  9  4 ...  9  9 17]


In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
#importing more libraries
import math

#downloading te right vocabulaty
from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor
bert_preprocessor = BertPreprocessor(vocab_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/vocab.txt",
                                     do_lower_case=False,
                                     max_seq_length=64)
from deeppavlov.models.bert.bert_classifier import BertClassifierModel


[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [36]:
from deeppavlov.models.bert.bert_classifier import BertClassifierModel
from deeppavlov.metrics.accuracy import sets_accuracy
bert_classifier = BertClassifierModel(
    n_classes=15,
    return_probas=False,
    one_hot_labels=True,
    bert_config_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_config.json",
    pretrained_bert="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt",
    save_path="ru_bert_model/model",
    load_path="ru_bert_model/model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

Using TensorFlow backend.






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use standard file APIs to check for files with this prefix.


2019-11-21 16:38:21.545 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 98: [initializing model with Bert from /Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt]



INFO:tensorflow:Restoring parameters from /Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt


In [37]:
#changing vocabulary format
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
vocab = SimpleVocabulary(save_path="./binary_classes.dict")

2019-11-21 16:54:30.704 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


In [38]:
print(vocab.items())

#input_features = bert_preprocessor([""])
from deeppavlov.models.bert.bert_classifier import BertClassifierModel
from deeppavlov.metrics.accuracy import sets_accuracy
bert_classifier = BertClassifierModel(
    n_classes=15,
    return_probas=False,
    one_hot_labels=True,
    bert_config_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_config.json",
    pretrained_bert="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt",
    save_path="ru_bert_model/model",
    load_path="ru_bert_model/model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

In [40]:
bert_classifier(bert_preprocessor(top_tokens_vec))

array([11, 11, 14, 11, 11, 11, 11, 11, 11, 11, 11, 14, 11, 11, 11])

# Turning BERT classifier into pretrained PyTorch

In [ ]:
from pytorch_transformers import BertForPreTraining, BertConfig, BertTokenizer, load_tf_weights_in_bert, BertModel

In [ ]:
# transformin deeppavlov BERT model into PyTorch pretrained model

def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file):
    config = BertConfig.from_json_file(bert_config_file)
    model = BertForPreTraining(config)
    model = load_tf_weights_in_bert(model, config, tf_checkpoint_path)

    return model

BERT_MODEL_PATH = "/Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/"
tf_checkpoint_path = BERT_MODEL_PATH + 'bert_model.ckpt'
bert_config_file = BERT_MODEL_PATH + 'bert_config.json'

downloaded = convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=False)

In [ ]:
encoder = downloaded.bert

# TFIDF Classifier based on LentaRu dataset

In [ ]:
# uploading data
lentaRu = pd.read_csv(r'/Users/apple/Downloads/lenta-ru-news.csv')
lentaRuTxt = lentaRu['text']
lentaRu.info()

# importing all libraries needed
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

#scikit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

#defining classes stemmer and stop words
stemmer = SnowballStemmer('russian')
words = stopwords.words('russian')

In [ ]:
#cleaning the data from odd symbols, stand alone letters
#lowercasing every document
lentaRu['cleaned'] = lentaRu['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^а-яА-Я]",
                                                                                               " ",
                                                                                               str(x)).split() if i not in words]).lower())
#checking out if any cell is NA
lentaRu.isna()

In [ ]:
#getting rid of NAs
lentaRuWon = lentaRu.dropna()
lentaRuWon
print(lentaRuWon['tags'].unique())
print(lentaRuWon['topic'].unique())

In [ ]:
#partitioning the data in to test and train frames
X_train, X_test, y_train, y_test = train_test_split(lentaRuWon['cleaned'],
                                                    lentaRuWon.topic,
                                                    test_size=0.05)

In [ ]:
#creating pipeline
#create TFIDF vector for single words and bigrams
#selecting only good features for our vector
#choosing SVC algorithm
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words=words, sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [ ]:
#fitting the model into the pipeliine
model = pipeline.fit(X_train, y_train)

In [ ]:
#preparation
#getting instances
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']
#transforming features
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [ ]:
#checking out accuracy score of the model
print("accuracy score: " + str(model.score(X_test, y_test)))
#accuracy score: 0.8038830593120476

In [41]:
import dill as pickle

#with open('testing_model.pk', 'wb') as file:
#    pickle.dump(model, file)
with open('testing_model.pk','rb') as f:
    model = pickle.load(f)


In [42]:
lda_predict = model.predict(top_tokens_vec)
lda_predict = pd.DataFrame(lda_predict)
lda_result = pd.concat([top_tokens_vec, lda_predict], axis=1)

In [43]:
print(lda_result)

                                                    0         0
0   человека лет жизни года год цена сегодня жизнь...  Из жизни
1   сегодня день стоит гороскоп лишь люди жизни по...  Из жизни
2   день жизни каждый люди работу людей лет скольк...  Из жизни
3   лет грн человека доставка фото срок хочу кажды...       Мир
4   день жизнь жизни сегодня человека год сколько ...  Культура
5   жизнь хочу стоит лишь радуйся жизни года день ...       Мир
6   жизнь года любовь сегодня день люди году цена ...  Из жизни
7   людей люди жизнь любишь хочешь жизни любить зн...  Из жизни
8   руб года жизнь лет тыс день квартира человека ...  Из жизни
9   жизни люди человека чтo почему людей поэтому д...  Из жизни
10  люди года сегодня лет любовь стоит часто челов...  Из жизни
11  жизни каждый люди лишь руки любовь люблю цена ...  Из жизни
12  сегодня года жизни люди хочу группу лишь репос...  Из жизни
13  день жизни каждый лишь мир людям сделать нужен...  Из жизни
14  жизнь человека людей люди любовь сег

In [ ]:
artm_predict = model.predict(artmToks)
artm_predict = pd.DataFrame(artm_predict)
artm_result = pd.concat([artmToks, artm_predict], axis=1)